# Mittelfluss aus Investitionstätigkeit

## Einlesen der erforderlichen Paket und Daten

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data.csv', index_col=0)

## Einschränkungen für die Berechnung

Die meisten Schulaufgaben gehen von bloss zwei Positionen im Anlagevermögen aus:
Mobilien und Immobilien. In der Realtität besteht das Anlagevermögen aus einer Vielzahl von
Vermögnspositionen.

Trotzdem werden in den vorliegenden Beispielen zur Vereinfachung nur die
Positionen Mobilien und Immobilien betrachtet.

## Rekonstruktion der Abschreibungen auf Mobilien

Um die erforderlichen Berechnungen zu planen, lohnt es sich, den Ablauf der
Berechnungen in einem Flussdiagramm darzustellen.

In [21]:
def _check_wb_mob(ledger: pd.DataFrame) -> bool:
    wb_mob = ledger.index.intersection([1509, 1519, 1529, 1539, 1549])
    
    return len(wb_mob) > 0

In [22]:
def _check_wb_immo(ledger: pd.DataFrame) -> bool:
        
    return 1609 in ledger.index

In [15]:
def _get_total_absch(ledger: pd.DataFrame) -> float:
    """
    Get the total amount of Abschreibungen from the ledger.
    """
    konti = [i for i in range(6800, 6900)]
    total_absch = 0
    for konto in konti:
        if konto in ledger.index:
            total_absch += ledger.loc[konto, 'Abschluss'] * -1
    
    return total_absch

In [24]:
def _get_delta_wb_mob(ledger: pd.DataFrame) -> float:
    """
    Get the delta of the wb_mob from the ledger.
    """
    wb_mob = ledger.index.intersection([1509, 1519, 1529, 1539, 1549])
    
    if len(wb_mob) == 0:
        return 0
    
    delta_wb_mob = 0
    for konto in wb_mob:
        delta_wb_mob += ledger.loc[konto, 'Abschluss'] - ledger.loc[konto, 'Eroeffnung']
    
    return delta_wb_mob

In [17]:
def _get_delta_wb_immo(ledger: pd.DataFrame) -> float:
    delta_wb_immo = ledger.loc[1609, 'Abschluss'] - ledger.loc[1609, 'Eroeffnung']
    return delta_wb_immo

In [29]:
def _get_absch_mob(ledger: pd.DataFrame) -> float:
    absch = _get_total_absch(ledger)    
 
    if _check_wb_mob(ledger):
        absch_mob = _get_delta_wb_mob(ledger)
    else:
        absch_mob =  absch - _get_delta_wb_immo(ledger)
    return absch_mob

In [28]:
def _get_absch_mob_wb(ledger: pd.DataFrame) -> float:
    konti = ledger.index.intersection([1509, 1519, 1529, 1539, 1549])
    absch_mob = 0
    for konto in konti:
        absch_mob += ledger.loc[konto, 'Abschluss'] - ledger.loc[konto, 'Eroeffnung']
    return absch_mob * -1  # Negativ, weil es eine Erhöhung des WB ist

In [50]:
def _get_absch_immo(ledger: pd.DataFrame) -> float:
    absch = ledger.loc[6800, 'Abschluss'] * -1
    absch_mob = _get_absch_mob(ledger) * -1    
    absch_immo = absch - absch_mob    
    return absch_immo

In [33]:
def _get_invest_mob(ledger: pd.DataFrame) -> float:
    mob = ledger.index.intersection([1500, 1510, 1520, 1530, 1540])
    invest_mob = 0
    for m in mob:
        invest_mob += ledger.loc[m, 'Abschluss'] - ledger.loc[m, 'Eroeffnung']
        
    if not _check_wb_mob(ledger):
        invest_mob += _get_absch_mob(ledger)
        
    return invest_mob

In [48]:
def _get_invest_immo(ledger: pd.DataFrame) -> float:
    delta_immo = ledger.loc[1600, 'Abschluss'] - ledger.loc[1600, 'Eroeffnung']
    
    if not _check_wb_immo(ledger):
        delta_immo += _get_absch_immo(ledger)
    
    return delta_immo

In [56]:
def investitionsbereich(ledger: pd.DataFrame):
    investitionen = {}
    investitionen['Mobilien'] = _get_invest_mob(ledger)
    investitionen['Immobilien'] = _get_invest_immo(ledger)
    
    breite = 50
    titel = 'Mittelfluss aus Investitionen\n\n'
    
    for index, value in investitionen.items():
        mitte = breite - len(index) -len(str(value))
        breite_links = len(index) + mitte // 2
        breite_rechts = len(str(value)) + mitte // 2
        if breite_links + breite_rechts < breite:
            breite_links += 1
        titel += f"{index:<{breite_links}}{value:>{breite_rechts}}\n"
        
    mitte = breite - len('Nettoinvestitionen') - len(str(investitionen['Mobilien'] + investitionen['Immobilien']))
    breite_links = len('Nettoinvestitionen') + mitte // 2
    breite_rechts = len(str(investitionen['Mobilien'] + investitionen['Immobilien'])) + mitte // 2
    if breite_links + breite_rechts < breite:
        breite_links += 1
    inv = 'Nettoinvestitionen'
    inv_zahl = str(investitionen['Mobilien'] + investitionen['Immobilien'])
    titel += f"{inv:<{breite_links}}{inv_zahl:>{breite_rechts}}\n"
        
    print(titel)
    

In [57]:
investitionsbereich(df)

Mittelfluss aus Investitionen

Mobilien                                      20.0
Immobilien                                     0.0
Nettoinvestitionen                            20.0

